<a href="https://colab.research.google.com/github/eisbetterthanpi/transformer/blob/main/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title hf byte dataset me
!pip install -qU datasets # restart?
import torch
from torch.utils.data import IterableDataset

class StreamDataset(IterableDataset):
    def __init__(self, dataset, seq_len=129, buffer_size=1024):
        self.vocab_size = 256 # utf-8 # self.enc.n_vocab # gpt2:50257
        self.data = iter(dataset)
        self.seq_len, self.buffer_size = seq_len, buffer_size  # must be ≥ seq_len
        self.buffer = []  # token buffer
        self.fill_buffer()

    def fill_buffer(self):
        while len(self.buffer) < self.buffer_size:
            x = next(self.data)
            # print(x)
            self.buffer.extend(x['text'].encode("utf-8"))

    def __iter__(self):
        while True:
            if len(self.buffer) < self.seq_len: self.fill_buffer()
            if len(self.buffer) < self.seq_len: return # raise StopIteration
            x, self.buffer = self.buffer[:self.seq_len], self.buffer[self.seq_len:]
            yield torch.tensor(x, dtype=torch.int32) # uint8 int32

from datasets import load_dataset
name = 'Skylion007/openwebtext' if torch.cuda.is_available() else 'stas/openwebtext-10k'
dataset = load_dataset(name, split="train", streaming=True, revision='refs/convert/parquet', cache_dir="/content/hf")

seq_len = 128*1+1 # 128
buffer_size = seq_len*1
train_data = StreamDataset(dataset, seq_len, buffer_size) # train_data = StreamDataset(dataset["train"], seq_len, buffer_size)
# del dataset

from torch.utils.data.dataloader import DataLoader
batch_size = 64 if torch.cuda.is_available() else 16 #64 512
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=0)
# del train_data
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# https://github.com/facebookresearch/blt/blob/main/bytelatent/tokenizers/blt_tokenizer.py#L137
# def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.uint8)#, device=device)#.unsqueeze(0)
def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.int32, device=device).unsqueeze(0)
def decode(x): return bytes(x.tolist()).decode("utf-8", errors='replace') # replace ignore

# for x in train_loader:
#     print(x.shape, x)
#     break
# print(decode(x[0][:64]))


In [ ]:
# @title yelp data
# !pip install -qU datasets
from datasets import load_dataset
# # dataset = load_dataset("Yelp/yelp_review_full", split="train", streaming=True, revision='refs/convert/parquet', cache_dir="/content/hf")
train_dataset = load_dataset("Yelp/yelp_review_full", split="train", revision='refs/convert/parquet', cache_dir="/content/hf")
test_dataset = load_dataset("Yelp/yelp_review_full", split="test", revision='refs/convert/parquet', cache_dir="/content/hf")
# dataset = load_dataset("yelp_polarity") # yelp_polarity yelp_review_full

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.int32, device=device)#.unsqueeze(0)
def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.int32)#.unsqueeze(0)
def decode(x): return bytes(x.tolist()).decode("utf-8", errors='replace') # replace ignore

from torch.utils.data import Dataset
# from torch.utils.data import IterableDataset
class ByteDataset(Dataset):
# class ByteDataset(IterableDataset):
    def __init__(self, data):
        self.data = data
        # self.data = iter(data)
        # self.data = self.data_process(data) # list of int
        self.vocab_size = 256 # utf-8 # self.enc.n_vocab # gpt2:50257
        # self.num_classes = 5
        # set(dataset['label'])
        labels = sorted(list(set(data['label'])))
        self.num_classes = len(labels) #
        self.stoi = {ch:i for i,ch in enumerate(labels)}
        self.itos = {i:ch for i,ch in enumerate(labels)}
        self.max_len = 128
    # def data_process(self, data): # str 10780437
    #     # return torch.tensor([self.stoi.get(c) for c in data]) # list of int 4570571 # stoi.get(c,UNK_IDX)
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        xy = self.data[idx]
        x, y = xy['text'], xy['label']
        idx = torch.randint(max(1, len(x)-self.max_len+1), size=(1,))
        x = x[idx: idx+self.max_len]
        return encode(x), self.stoi[y]
        # return encode(xy['text']), xy['label']
    # def __iter__(self):
    #     while True:
    #         xy = next(self.data)
    #         yield encode(xy['text']), xy['label']

train_data = ByteDataset(train_dataset)
test_data = ByteDataset(test_dataset)

pad_idx = 0
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    x, y = zip(*batch)
    x = pad_sequence(x, batch_first=True, padding_value=pad_idx, padding_side='left')
    # print(x,y)
    return x, torch.tensor(y)#.unsqueeze(-1)

from torch.utils.data.dataloader import DataLoader
batch_size = 64 if torch.cuda.is_available() else 16 #64 512
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=0)

# for i,(x,y) in enumerate(train_loader):
#     # print(x.shape, x)
#     print(i, x, y)
#     break
# print(decode(x[3]))


In [ ]:
# o = decode(x[0])
# print(o)
# print(encode(o))
seq_len = 2**9+1 # 128
buffer_size = seq_len*1
train_data = StreamDataset(dataset, seq_len, buffer_size) # train_data = StreamDataset(dataset["train"], seq_len, buffer_size)
batch_size = 64
# train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=2)
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=0)
# for i, x in enumerate(train_loader):
#     # print(x.shape, x)
#     print(i, x.shape)
#     # break

# swa32 batch64 dim128 3lyr 2^10 3.5,7.3 55.4s
# swa32 2^11 4.3,14.5 110.5s        # swa32sig 2^11 5.9,14.7 110.5s
# swa16 2^11 4.3,9.8 85.3s
# swa8 2^11 4.4,7.2 77.4s
# swa8 batch16 2^13 5.1,7.2 77.3s


# mha 2^10 oom
# 2^9 3.5,4.7


# swa32sig
# this is what They knevestigh with the thes groses lechnines many in such is
# 11600 time: 6.5275774002075195 0.0005626737244241598
# strain 1.845391869544983
# this is what be of the picent strature by lorge, what do sest a pla know veh

# swa32sig attnnores
# this is what roub 201 to one dur. For the creates with wouldn’t the deeder
# 44800 time: 6.389352083206177 0.0001426162933872756
# strain 1.724169373512268
# this is what did thouldled beaked U.S. hendlife, the women concess of grow,



In [ ]:
# @title random_masking
import torch

def random_masking(length, mask_ratio=.75, b=64):
    noise = torch.rand(b, length)
    len_mask = int(length * mask_ratio)
    _, msk_ind = torch.topk(noise, k=len_mask, dim=-1, sorted=False) # val, ind -> [b,len_mask]
    _, keep_ind = torch.topk(noise, k=length-len_mask, largest=False, dim=-1, sorted=False) # val, ind -> [b,len_keep]
    return msk_ind, keep_ind

# msk_ind, keep_ind = random_masking(10, .3, b=2)

# x_ = torch.rand(4, 3, 2)
# print(x_)
# # ids = torch.tensor([0, 2, 1])[None,:,None]
# # ids = torch.tensor([0, 2, 1])[None,:,None].repeat(4,1,2)
# ids = torch.tensor([1, 2, 0])[None,:,None].repeat(4,1,2)
# # o = torch.gather(x_, dim=1, index=ids)
# o = torch.zeros_like(x_).scatter_(dim=1, index=ids, src=x_)
# print(o)


In [ ]:
# @title RoPE
# import torch
# import torch.nn as nn
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# class RoPE(nn.Module): # Rotary Positional Embeddings
#     def __init__(self, dim, seq_len=512, base=10000):
#         super().__init__()
#         theta = 1.0 / (base ** (torch.arange(0, dim, step=2) / dim))
#         pos = torch.arange(seq_len).unsqueeze(1)
#         angles = (pos * theta).unsqueeze(-1) # [seq_len, 1] * [dim // 2] -> [seq_len, dim // 2, 1]
#         self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).to(device) # [seq_len, dim // 2, 2]

#     def forward(self, x):
#         batch, seq_len, dim = x.shape
#         if seq_len > self.rot_emb.shape[0]: self.__init__(dim, seq_len)
#         rot_emb = self.rot_emb[:seq_len].unsqueeze(0).expand(batch, -1, -1, -1) # [batch, seq_len, dim//2, 2]
#         x = x.reshape(batch, seq_len, dim // 2, 2)
#         rot_x = x * rot_emb
#         return rot_x.flatten(-2)

# @title RoPE
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RoPE(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, seq_len=512, base=10000):
        super().__init__()
        self.dim, self.base = dim, base
        theta = 1.0 / (base ** (torch.arange(0, dim, step=2) / dim))
        pos = torch.arange(seq_len).unsqueeze(-1)
        angles = (pos * theta)[None,...,None] # [seq_len, 1] * [dim // 2] -> [1, seq_len, dim // 2, 1]
        self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [seq_len, dim // 2, 2] -> [1, seq_len, dim]

    def forward(self, x): # [b,t,d]
        seq_len = x.size(1)
        if self.rot_emb.shape[1] < seq_len: self.__init__(self.dim, seq_len, self.base)
        return x * self.rot_emb[:,:seq_len]
        # return x + self.rot_emb[:,:seq_len]


# dim=16
# seq_len=512
# rope = RoPE(dim, seq_len, base=10000)
# x = torch.rand(4,64,dim, device=device)
# out = rope(x)
# print(out.shape)


# def RoPE(dim, seq_len=512, base=10000):
#     theta = 1.0 / (base ** (torch.arange(0, dim, step=2) / dim))
#     pos = torch.arange(seq_len).unsqueeze(-1)
#     angles = (pos * theta)[None,...,None] # [seq_len, 1] * [dim // 2] -> [1, seq_len, dim // 2, 1]
#     rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [1, seq_len, dim // 2, 2] -> [1, seq_len, dim]
#     return rot_emb

# context_indices, trg_indices = random_masking(10, .3, b=1)


# # [1,t,d]
# # [b,num_msk]
# if mask_indices != None: q = q* rope[torch.arange(b).unsqueeze(-1), mask_indices] # [batch, num_context_toks, d_model]



In [ ]:
# @title complement
import torch
def complement(x, length): # [b,t], int
    b = x.shape[0]
    device = x.device
    full = torch.arange(length, device=device).repeat(b,1)   # (length,)
    mask = torch.ones(b, length, dtype=bool, device=device)
    mask[torch.arange(b).unsqueeze(-1), x] = False
    return full[mask].reshape(b,-1)



def batch_missing(x: torch.Tensor, n: int):
    b, t = x.shape
    device = x.device
    presence = torch.zeros((b, n), dtype=torch.bool, device=device)
    presence.scatter_(1, x, True)
    missing_mask = ~presence  # shape [b, n]
    all_indices = torch.arange(n, device=device).expand(b, n)
    missing = all_indices[missing_mask].view(b, n - t)
    return missing


def batch_missing_fast(x: torch.Tensor, n: int):
    b, t = x.shape
    device = x.device
    full = torch.arange(n, device=device).expand(b, n)  # [b, n]
    mask = torch.ones((b, n), dtype=torch.bool, device=device)
    mask.scatter_(1, x, False)  # Mark present values as False
    filled = full.masked_fill(~mask, n)
    sorted, _ = filled.sort(dim=1)
    return sorted[:, :n - t]


def complement(x, t): # [b,num_mask]
    counts = torch.bincount(x.flatten(), minlength=t)
    missing_values = torch.nonzero(counts==0, as_tuple=False).squeeze(1)
    return missing_values


b = 2
msk_len=4
x = torch.stack([torch.randperm(10)[:msk_len] for _ in range(b)]) # [b,msk_len]
print(x)
print(complement(x,10))


In [ ]:
# @title Sliding Window Attention
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

class Attention(nn.Module): # sliding window attention
    # def __init__(self, d_model, cond_dim=None, n_heads=None, d_head=8, drop=0.): # .1
    def __init__(self, query_dim, cond_dim=None, n_heads=8, d_head=8, drop=0, w=32):
        super().__init__()
        self.d_model = d_model = d_head * n_heads
        self.d_head, self.n_heads = d_head, n_heads
        self.cond_dim = cond_dim
        self.pos_enc = RoPE(d_model, base=10000) # 10000
        self.q = nn.Linear(query_dim, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or query_dim, 2*d_model, bias=False)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(drop) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5
        # torch.nn.init.normal_(self.qkv.weight, std=.02)
        # torch.nn.init.normal_(self.q.weight, std=1/(math.sqrt(query_dim)+math.sqrt(d_model)))
        # torch.nn.init.normal_(self.kv.weight, std=1/(math.sqrt(cond_dim or query_dim)+math.sqrt(d_model)))
        self.w = w

    def forward(self, x, cond=None, mask=None): # [b,t,d], [batch, num_tok, cond_dim], [b,t]
        b,t = x.shape[:2]
        if self.cond_dim==None: cond=x # is self attn
        q = self.pos_enc(self.q(x)).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        k, v = self.kv(cond).chunk(2, dim=-1)
        k = self.pos_enc(k).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, n_heads, T/num_tok, d_head]
        v = v.unflatten(-1,(self.n_heads, self.d_head)).transpose(1,2) # [batch, n_heads, T/num_tok, d_head]
        # k, v = F.pad(k,(0,0,self.w-1,0)), F.pad(v,(0,0,self.w-1,0)) # causal # [b, h, t+w-1, 2*d]
        k, v = F.pad(k,(0,0,int(self.w/2),(self.w-1)//2)), F.pad(v,(0,0,int(self.w/2),(self.w-1)//2)) # bidirectional # [b, h, t+w-1, 2*d]
        k, v = k.unfold(-2,self.w,1).transpose(-2,-1), v.unfold(-2,self.w,1).transpose(-2,-1) # [b,h,t,w,d]

        attn = torch.einsum("bhtd,bhtwd->bhtw", q, k) * self.scale
        # print('attn fwd q mk attn', q.dtype, mk.dtype, attn.dtype)
        if mask != None: attn = attn.masked_fill(~mask[:,None,:,None], -torch.finfo(attn.dtype).max) # [b,t]->[b,1,t,1] # causal is built into swa, so mask is only for [b,t]
        attn = torch.softmax(attn, dim=-1) # [b,h,t,1,w]
        # attn = F.sigmoid(attn-math.log(attn.shape[-1])) # https://arxiv.org/pdf/2409.04431
        # out = (self.drop(attn) @ mv).squeeze(-2) # [b,h,t,1,w]@[b,h,t,w,d]=[b,h,t,1,d]
        out = torch.einsum("bhtw,bhtwd->bhtd", self.drop(attn), v)
        out = out.transpose(1, 2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]

# typically: gen td*td=tt
# up t1d*twd=tw, patch pd*tpd=p
# down td*(w+1)d=t*(w+1)

# t(2d+t)
# t((w+1)d + w)
# d+t >= wd+w
# d~64-512; w~32-256; t~128*4^3=8192=2^13
# 2^9; 2^8

# b,t,d = 64,100,512

# mask = torch.rand(b,t, device=device)>.5
# model = Attention(d, w=3).to(device) # 257 ms
# x = torch.rand(b,t,d, device=device)
# out = model(x, mask=mask)


In [ ]:
# @title MultiHeadAttention
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

# class MultiHeadAttention(nn.Module):
class Attention(nn.Module):
    # def __init__(self, d_model, n_heads=None, d_head=8, cond_dim=None, drop=0.): # .1
    def __init__(self, query_dim, cond_dim=None, n_heads=8, d_head=64, drop=0):
        super().__init__()
        # self.d_model, self.n_heads, self.d_head = d_model, n_heads, d_model // n_heads
        d_model = d_head * n_heads
        self.d_head, self.n_heads = d_head, n_heads
        self.cond_dim = cond_dim
        self.pos_enc = RoPE(d_model, base=10000) # 10000
        self.q = nn.Linear(query_dim, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or d_model, 2*d_model, bias=False)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(drop) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5
        # torch.nn.init.normal_(self.q.weight, std=.02)
        # torch.nn.init.normal_(self.kv.weight, std=.02)
        # torch.nn.init.normal_(self.q.weight, std=1/(math.sqrt(query_dim)+math.sqrt(d_model)))
        # torch.nn.init.normal_(self.kv.weight, std=1/(math.sqrt(cond_dim or query_dim)+math.sqrt(d_model)))

    def forward(self, x, cond=None, mask=None): # [batch, T, d_model]=[batch, h*w, c], [batch, num_tok, cond_dim], [batch,T]
        # print('att fwd', x.shape, cond.shape if cond!=None else None, mask.shape if mask!=None else None)
        if self.cond_dim==None: cond=x # is self attn
        # q = self.q(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        # # K = self.k(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2)
        # k, v = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2).chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]

        q = self.pos_enc(self.q(x)).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        k, v = self.kv(cond).chunk(2, dim=-1)
        k = self.pos_enc(k).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2)
        v = v.unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2)

        # # (quadratic) attention # Softmax(q @ k.T) @ v
        out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask.unsqueeze(1) if mask != None else None, dropout_p=0) # mask: [batch,len_q, len_v]
        # # out = F.scaled_dot_product_attention(q, k, v, is_causal=True, dropout_p=0) # mask: [batch,len_q, len_v]
        # attn = q @ k.transpose(-2,-1) * self.scale # [batch, n_heads, T] # [batch, n_heads, T, T/num_tok]
        # if mask != None: attn = attn.masked_fill(~mask.unsqueeze(1), -torch.finfo(attn.dtype).max) # [b,t,t]->[b,1,t,t]
        # attn = torch.softmax(attn, dim=-1)
        # # attn = F.sigmoid(attn-torch.log(attn.shape[-1])) # https://arxiv.org/pdf/2409.04431
        # out = self.drop(attn) @ v # [batch, n_heads, T, d_head]

        out = out.transpose(1,2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]



In [ ]:
# @title Attention
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class SwiGLU(nn.Module): # https://arxiv.org/pdf/2002.05202
    def __init__(self, d_model, ff_dim): # d_model * 3*ff_dim params
        super().__init__()
        self.lin0 = nn.Linear(d_model, 2*ff_dim, bias=False)
        self.lin1 = zero_module(nn.Linear(ff_dim, d_model, bias=False))
        # torch.nn.init.normal_(self.lin0.weight, std=.02)
        torch.nn.init.normal_(self.lin0.weight, std=1/(math.sqrt(d_model)+math.sqrt(ff_dim)))

    def forward(self, x): # [b,t,d]
        x0, x1 = self.lin0(x).chunk(2, dim=-1)
        return self.lin1(x0*F.silu(x1))

# 2048*2
# 2048*7
# ff: d_model*2 *ff_dim params

class AttentionBlock(nn.Module):
    # def __init__(self, d_model, cond_dim=None, d_head, ff_dim=None, drop=0.):
    def __init__(self, d_model, n_heads ,cond_dim=None, ff_dim=None, drop=0):
        super().__init__()
        self.d_model = d_model
        self.cond_dim = cond_dim
        self.norm1 = nn.RMSNorm(d_model) # LayerNorm RMSNorm
        if cond_dim!=None: self.norm2 = nn.RMSNorm(cond_dim)
        # self.drop = nn.Dropout(drop)
        self.attn = Attention(d_model, cond_dim or d_model, n_heads=n_heads, d_head=d_model//n_heads, drop=drop)
        act = nn.SiLU() # GELU SiLU ReLU
        if ff_dim==None: ff_dim=d_model*4
        self.ff = nn.Sequential(
            nn.RMSNorm(d_model), nn.Linear(d_model, ff_dim), act,
            nn.RMSNorm(ff_dim), nn.Dropout(drop), zero_module(nn.Linear(ff_dim, d_model))
            # nn.RMSNorm(d_model), act, nn.Linear(d_model, ff_dim),
            # nn.RMSNorm(ff_dim), act, zero_module(nn.Linear(ff_dim, d_model))
        )
        # torch.nn.init.normal_(self.ff[1].weight, std=.02)
        torch.nn.init.normal_(self.ff[1].weight, std=1/(math.sqrt(d_model)+math.sqrt(ff_dim)))
        # self.ff = SwiGLU(d_model, ff_dim)

    def forward(self, x, cond=None, mask=None): # [b,c,h,w], [batch, num_tok, cond_dim], [batch,T]
        # print('attblk fwd', x.shape, cond.shape if cond!=None else None, mask.shape if mask!=None else None)
        # if self.cond_dim==None: x = x + self.attn(self.norm1(x), mask=mask)
        # else: x = x + self.attn(self.norm1(x), self.norm2(cond), mask) # maybe no res for decoder
        x = self.attn(self.norm1(x), self.norm2(cond)if cond!=None else self.norm1(x), mask) #
        x = x + self.ff(x) # maybe no ff for decoder?
        return x

import inspect
class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
        for layer in self:
            params = inspect.signature(layer.forward).parameters.keys()
            layer._fwdparams = ','.join(params)

    def forward(self, x, cond=None, mask=None):
        for layer in self:
            args = [x]
            if 'cond' in layer._fwdparams: args.append(cond)
            if 'mask' in layer._fwdparams: args.append(mask)
            x = layer(*args)
        return x

b,t,d = 2,5,16
# b,t,d = 2,5,256
x = torch.rand(b,t,d, device=device)
mask = torch.rand(b,t,t, device=device)>0
model = AttentionBlock(d_model=d, n_heads=4, ff_dim=16).to(device)
# model = nn.Sequential(*[AttentionBlock(d_model=d, n_heads=4, ff_dim=16) for _ in range(2)])
out =  model(x, mask=mask)
# out =  model(x)
print(out.shape)


# # model = MultiHeadAttention(d).to(device) # 257 ms
# x = torch.rand(b,t,d, device=device)

# import time
# start = time.time()
# # out = model(x)
# out = model(x, mask=mask)
# print(out.shape)
# print(time.time()-start)



torch.Size([2, 5, 16])


In [5]:
import torch
import torch.nn as nn

class AttentionBlock(nn.Module):
    def __init__(self, i):
        super().__init__()
        self.i=i

    def forward(self, x, cond=None):
        print('attblk fwd', x, cond)
        x=x+self.i+1
        print(x)
        return x

import inspect
class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
        for layer in self:
            params = inspect.signature(layer.forward).parameters.keys()
            layer._fwdparams = ','.join(params)

    def forward(self, x, cond=None, mask=None):
        for layer in self:
            args = [x]
            if 'cond' in layer._fwdparams: args.append(cond)
            if 'mask' in layer._fwdparams: args.append(mask)
            x = layer(*args)
        return x

n_layers = 2
enc = Seq(*[AttentionBlock(i) for i in range(n_layers)])
x=3
c=2
out = enc(x)
out = enc(x,c)




attblk fwd 3 None
4
attblk fwd 4 None
6
attblk fwd 3 2
4
attblk fwd 4 2
6


In [ ]:
# @title GPT
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GPT(nn.Module):
    def __init__(self, in_dim, d_model=64, out_dim=None, n_heads=8, n_layers=1, ff_dim=256, dropout=0):
        super().__init__()
        # self.d_model = d_model
        # self.pos_enc = RoPE(d_model, base=10000)
        self.tok_emb = nn.Embedding(in_dim+1, d_model)
        # self.tok_emb = TTEmbedding([29, 1733], [64,1], rank=min(d_model,256))
        # self.encoder = Seq(*[AttentionBlock(d_model, n_heads=n_heads, cond_dim=d_model) for _ in range(n_layers)])
        self.encoder = Seq(*[AttentionBlock(d_model, n_heads=n_heads) for _ in range(n_layers)])
        # self.out = nn.Linear(d_model, out_dim)
        # self.out = lambda x: x @ self.tok_emb.weight().T  # weight tying
        self.out = lambda x: x @ self.tok_emb.weight[:-1].T # [vocab+1,d]->[d,vocab] # weight tying for nn.Embedding

    def forward(self, x, mask=None): # [b,t], [b,t,t]
        # x = self.pos_enc(x)
        x = self.tok_emb(x)
        # print('gpt fwd', x.shape)
        b,t,d = x.shape

        causal_mask = torch.tril(torch.ones((b,t,t), dtype=bool, device=device)) # [b,q,cond]
        if mask!=None: causal_mask = causal_mask & mask.unsqueeze(-2) & mask.unsqueeze(-1) # [b,t,t]

        x = self.encoder(x, mask=causal_mask) # [b,t,d], [b,t,t]
        x = self.out(x)
        return x

# gpt 2
# Parameters Layers dmodel
# 117M 12 768 gpt1
# 345M 24 1024
# 762M 36 1280
# 1542M 48 1600


# attnnores improve fast but slows down, gets overtaken
# sigmoid ~< softmax


try: vocab_size=train_loader.dataset.vocab_size#50
except NameError: vocab_size=50
# d1024 12lyr h16, 768 8 12, 512 6 8, 256 4 8, , 64 1 8
# 768 16 12, 768 12 18
# model = GPT(input_size, d_model=512, out_dim=num_classes, n_layers=6).to(device)
# model = GPT(vocab_size, d_model=512, out_dim=vocab_size, n_layers=3, n_heads=16).to(device)
model = GPT(vocab_size, d_model=256, out_dim=vocab_size, n_layers=2, n_heads=8).to(device)
# model = GPT(vocab_size, d_model=64, out_dim=vocab_size, n_layers=1, n_heads=8).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
# optim = torch.optim.AdamW(model.parameters(), 1e-3) # og betas=(0.9, 0.999), wd.01
# optim = torch.optim.AdamW(model.parameters(), 1e-3, betas=(0.9, 0.95), weight_decay=1e-1) # og wd.01
optim = torch.optim.AdamW(model.parameters(), 1e-3, weight_decay=1e-1) # og wd.01

# x = torch.randint(0, vocab_size, (64, 128), device=device)
# out = model(x)
# print(out.shape)
# print(out)

# strain 5.660660743713379
# this is what we've done it. I'm able to be something strong. I think their information can be highly specific to a busy intertwined. But why "Effects"/ who shouldn't do," suppose it would improve them.Earlier open in both countries is sent by documentation, who were involved by custom ways to cutting the carbon gas
# strain 5.939525604248047
# this is what I’d like he’s very important to learn, it’s a bug my liquid had nothing once his streaming user in their own. It’s enough to run the survey, if there’s also no very big interested in a very $2. It’s sufficient's
# strain 5.96616792678833
# this is what manyatform are neither even documented and looked down their intentions in writers in so far savigTS-ve nuite was never known that some dioxide the sizes celebrate was fighting, as guilty or innovation.672 Expressia lives to Islamic Cheney of your existence takes a free background rest: The book she probably read it or find
# strain 5.811507225036621
# this is what it has not seen in intelligence and have believed in a romantic order and possible and rapidly now. Many growing economic ear is unclear whether they can do that the country's job. According to a proposals, the majority of domestic trade on which are the greatest mission with rich law by people, violated stockrol, which they would


# b64,l128 8.0ram,10.0gpu l128*2:oom

# b64,l128 w64 8.2ram,6.8gpu
# b64,l128*2 w64 8.1ram,14.0gpu

# gpt 1lyr mha ropein100 @attn # low loss but not learning
# gpt 1lyr mha ropeout10000 F.attn # 19.3s
# gpt 1lyr mha ropein10000 @attn # not learning
# gpt 1lyr mha ropein10000 F.attn # this is what you are something else all in course, you're simply went back over their news, the reaction of the French source that is proven as we did not change any life at each power because they have the which shall noteded in order to change.

# gpt 1lyr mha ropein10000 @attn fix~mask # 18.5s 7.3,5.9

# swa64 128*2 7.4,12.0

# d256 6.9,6.7

# dim128 3lyr 3.0,.7
# this is whats more of ietre'soni, they asari, your very
# they vate. This iat
# 9600 time: 5.7726781368255615 0.0006012580827081866
# strain 1.481871247291565
# this is what line wasn't good to both feel
# like a delieve able or so foll,

# mha
# seq128*4+1 3.0,4.3
# seq128*8+1 oom

# swa
# seq128*4+1 w64: 2.4,9.1 crash? w32:2.5,3.8
# this is whatusanintan]
#  dacrstan mowainind Butun
#  an����������������������
# 1500 time: 39.80479311943054 0.026518849830957827
# strain 2.2812118530273438
# this is whathe oue suses
# catedeintilly,27), afevonees, Whe theesereceempone

# seq128*8+1 w64:oom w32:2.4,7.8





1644800


In [ ]:
# @title gpt-bert
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GPT(nn.Module):
    def __init__(self, in_dim, d_model=64, out_dim=None, n_heads=8, n_layers=1, ff_dim=256, dropout=0):
        super().__init__()
        self.d_model = d_model
        self.pos_enc = RoPE(d_model, base=10000)
        self.tok_emb = nn.Embedding(in_dim+1, d_model) # +1 for msk_idx
        self.encoder = Seq(*[AttentionBlock(d_model, n_heads=n_heads, cond_dim=d_model) for _ in range(n_layers)])
        # self.out = lambda x: x @ self.tok_emb.weight.T  # weight tying
        self.out = lambda x: x @ self.tok_emb.weight[:-1].T  # weight tying
        # print(self.tok_emb.weight.shape) # [257, 64]

    def forward(self, x, mask=None): # [b,t], [b,t,t]
        # x = self.pos_enc(self.tok_emb(x))
        x = self.tok_emb(x)
        b,t,d = x.shape

        # causal_mask = torch.tril(torch.ones((b,t,x.shape[1]), dtype=bool, device=device)) # [1,t,n_cond+n_patch] # got cond, all can attend to cond
        # mask = causal_mask * mask.unsqueeze(1) if mask!=None else causal_mask # *[b,1,t] # mask left side, tril is lower left

        # x = self.encoder(x, mask=mask) # [b,t,d], [nlyr,b,d]
        x = self.encoder(x, x, mask=mask) # [b,t,d], [nlyr,b,d]
        x = self.out(x)
        return x

    def bert_loss(self, x, mask_ratio=.15, m=32): # [b,t]
        b,t = x.shape
        mask = (torch.rand(m,t) < mask_ratio).repeat(b,1) # [b*m,t]
        x = x.repeat_interleave(m, dim=0) # [b*m,t]
        y = x[mask]
        x[mask] = msk_idx
        y_ = model(torch.cat([torch.full((b*m,1), msk_idx, device=device), x], dim=-1))[:,:-1] #output = [batch size, trg len - 1, output dim]
        loss = F.cross_entropy(y_[mask], y.to(int)) # [b*t,d], [b*t]
        return loss


# gpt 2
# Parameters Layers dmodel
# 117M 12 768 gpt1
# 345M 24 1024
# 762M 36 1280
# 1542M 48 1600

msk_idx = 256
try: vocab_size=train_loader.dataset.vocab_size#50
except NameError: vocab_size=50
# model = GPT(input_size, d_model=512, out_dim=num_classes, n_layers=6).to(device)
# model = GPT(vocab_size, d_model=128, out_dim=vocab_size, n_layers=3).to(device)
model = GPT(vocab_size, d_model=64, out_dim=vocab_size, n_layers=1).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)

# x = torch.randint(0, vocab_size, (64, 128), device=device)
# # out = model(x)
# out = model.bert_loss(x)
# print(out.shape)
# print(out)

out_dim=train_data.num_classes
class Classifier(nn.Module):
    def __init__(self, in_dim, num_classes=10):
        super().__init__()
        self.classifier = nn.Linear(in_dim, num_classes)
    def forward(self, x): return self.classifier(x)
classifier = Classifier(vocab_size, out_dim).to(device)
coptim = torch.optim.SGD(classifier.parameters(), lr=1e-3) # 1e-3



In [ ]:
# @title gpt-bert
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GPT(nn.Module):
    def __init__(self, in_dim, d_model=64, out_dim=None, n_heads=8, n_layers=1, ff_dim=256, dropout=0):
        super().__init__()
        self.d_model = d_model
        self.pos_enc = RoPE(d_model, base=10000)
        self.tok_emb = nn.Embedding(in_dim+1, d_model) # +1 for msk_idx
        self.encoder = Seq(*[AttentionBlock(d_model, n_heads=n_heads, cond_dim=d_model) for _ in range(n_layers)])
        self.decoder = Seq(*[AttentionBlock(d_model, n_heads=n_heads, cond_dim=d_model) for _ in range(n_layers)])
        # self.out = lambda x: x @ self.tok_emb.weight.T  # weight tying
        self.out = lambda x: x @ self.tok_emb.weight[:-1].T  # weight tying
        # print(self.tok_emb.weight.shape) # [257, 64]


    def forward(self, x, context_indices, trg_indices): # [batch, seq_len, d_model], [batch, seq_len] # True will be ignored by the attention # https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html
    def forward(self, x, mask=None): # [b,t], [b,t,t]
        # x = self.pos_enc(self.tok_emb(x))
        x = self.tok_emb(x)
        b,t,d = x.shape

        if context_indices != None: x = x[torch.arange(x.shape[0]).unsqueeze(-1), context_indices] # [batch, num_context_toks, d_model]

        # causal_mask = torch.tril(torch.ones((b,t,x.shape[1]), dtype=bool, device=device)) # [1,t,n_cond+n_patch] # got cond, all can attend to cond
        # mask = causal_mask * mask.unsqueeze(1) if mask!=None else causal_mask # *[b,1,t] # mask left side, tril is lower left

        # x = self.encoder(x, mask=mask) # [b,t,d], [nlyr,b,d]
        x = self.encoder(x, x, mask=mask) # [b,t,d], [nlyr,b,d]

        x = self.decoder(x)

        x = self.out(x)
        return x

    def bert_loss(self, x, mask_ratio=.15, m=32): # [b,t]
        b,t = x.shape
        mask = (torch.rand(m,t) < mask_ratio).repeat(b,1) # [b*m,t]
        x = x.repeat_interleave(m, dim=0) # [b*m,t]
        y = x[mask]
        x[mask] = msk_idx
        y_ = model(torch.cat([torch.full((b*m,1), msk_idx, device=device), x], dim=-1))[:,:-1] #output = [batch size, trg len - 1, output dim]
        loss = F.cross_entropy(y_[mask], y.to(int)) # [b*t,d], [b*t]
        return loss


mae/ijepa enc:trans norm (lin if out!=dim); dec:trans norm lin


# gpt 2
# Parameters Layers dmodel
# 117M 12 768 gpt1
# 345M 24 1024
# 762M 36 1280
# 1542M 48 1600

msk_idx = 256
try: vocab_size=train_loader.dataset.vocab_size#50
except NameError: vocab_size=50
# model = GPT(input_size, d_model=512, out_dim=num_classes, n_layers=6).to(device)
# model = GPT(vocab_size, d_model=128, out_dim=vocab_size, n_layers=3).to(device)
model = GPT(vocab_size, d_model=64, out_dim=vocab_size, n_layers=1).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)

# x = torch.randint(0, vocab_size, (64, 128), device=device)
# # out = model(x)
# out = model.bert_loss(x)
# print(out.shape)
# print(out)

out_dim=train_data.num_classes
class Classifier(nn.Module):
    def __init__(self, in_dim, num_classes=10):
        super().__init__()
        self.classifier = nn.Linear(in_dim, num_classes)
    def forward(self, x): return self.classifier(x)
classifier = Classifier(vocab_size, out_dim).to(device)
coptim = torch.optim.SGD(classifier.parameters(), lr=1e-3) # 1e-3



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# @title wandb
!pip install -q wandb
import wandb # https://docs.wandb.ai/quickstart
wandb.login(key='487a2109e55dce4e13fc70681781de9f50f27be7')
try: run.finish()
except NameError: pass
run = wandb.init(project="gpt", config={"model": "res18",}) #

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train loss,█▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
train loss,0.05021


In [ ]:

train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=0)

for i,(x,y) in enumerate(train_loader):
    # print(x.shape, x)
    print(i, x, y)
    break
# print(decode(x[3]))
x = x[:0]

mask_ratio=.25
m=1
mask = (torch.rand(m,t) < mask_ratio).repeat(b,1) # [b*m,t]
x = x.repeat_interleave(m, dim=0) # [b*m,t]
y = x#[mask]
x[mask] = msk_idx
output = model(torch.cat([torch.full((b*m,1), msk_idx, device=device), x], dim=-1))[:,:-1] #output = [batch size, trg len - 1, output dim]
temperature=1
output = output/temperature
output = F.softmax(output, dim=-1) # vocab_size to char
ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
# print(y)
print(decode(y))
x[mask] = ix
# print(x)
print(decode(x))



In [ ]:
# @title bert train test
import torch
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

# # https://www.comet.com/site/blog/perplexity-for-llm-evaluation/
# def Perplexity(logits, target): # [b,t,vocab_size], [b,t]
#     log_probs = F.log_softmax(logits, dim=-1)
#     nll = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1) # [b,t]
#     perplexity = nll.mean().exp()
#     return perplexity

import time
def strain(model, dataloader, optim): # train function with automatic mixed precision
    model.train()
    start = begin = time.time()
    for i, (x,_) in enumerate(dataloader):
        x = x.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            b, t = x.shape
            # print(x.shape)
            loss = model.bert_loss(x)
            # # loss = F.cross_entropy(logits.flatten(0,1), y.flatten().to(int)) # [b*t,d], [b*t]
        optim.zero_grad()
        scaler.scale(loss).backward()
        # scaler.unscale_(optim)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optim)
        scaler.update()
        # if scheduler is not None: scheduler.step()
        if i % 100 == 0:
            print("strain",loss.item())
            # print(generate(model, "this is what"))
            # model.train()
            # perplexity = Perplexity(logits.detach(), y).item()
            print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
            start = time.time()
        try: wandb.log({"train loss": loss.item()/len(x)})
        except NameError: pass
        if i>=500: break

def ctrain(model, classifier, dataloader, coptim, scheduler=None): # train function with automatic mixed precision
    model.eval()
    classifier.train()
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device) # [batch, ]
        with torch.autocast(device_type=device, dtype=torch.bfloat16): # bfloat16 float16
            with torch.no_grad():
                # sx = model(x).detach()
                sx = model(torch.cat([torch.full((x.shape[0],1), msk_idx, device=device), x], dim=-1))[:,:-1].mean(dim=-2).detach() # [b,t,d]->[b,d]
            y_ = classifier(sx) # [b,vocab]
            loss = F.cross_entropy(y_, y.flatten().to(int)) # [b,vocab], [b]
        coptim.zero_grad()
        scaler.scale(loss).backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 5) # .5
        scaler.step(coptim)
        scaler.update()
        if i % 100 == 0:
            print("classify",loss.item()/len(y))
        try: wandb.log({"closs": loss.item()/len(y)})
        except NameError: pass
        if i>=100: break

def test(model, classifier, dataloader):
    model.eval()
    classifier.eval()
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device) # [batch, ]
        with torch.no_grad():
            # sx = model(x)
            sx = model(torch.cat([torch.full((x.shape[0],1), msk_idx, device=device), x], dim=-1))[:,:-1].mean(dim=-2).detach() # [b,t,d]->[b,d]
            y_ = classifier(sx)

        loss = F.cross_entropy(y_, y.flatten().to(int)) # [b,vocab], [b]
        # acc = ((y.flatten()==y_.argmax(dim=1)).sum()/len(y)).item()
        acc = ((y==y_.argmax(dim=1)).sum()/len(y)).item()
        # print(y.shape, y_.shape, y_.argmax(dim=1).shape)
        # print((y.flatten()==y_.argmax(dim=1)))
        print
        # if i>=100: break
        if i % 100 == 0:
            # print("test",loss.item()/len(y))
            print('acc', round(acc, 3), 'test_loss', round(loss.item()/len(y), 3))
        try: wandb.log({"correct": acc})
        # try: wandb.log({"test loss": loss.item()/len(y)})
        except NameError: pass
        # if i>=100: break
        if i>=0: break

# scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=400, num_training_steps=2000) # https://docs.pytorch.org/torchtune/0.2/generated/torchtune.modules.get_cosine_schedule_with_warmup.html
for i in range(10): #
    train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=0)
    test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, pin_memory=True, num_workers=0)

    strain(model, train_loader, optim)
    ctrain(model, classifier, train_loader, coptim)
    test(model, classifier, test_loader)


In [ ]:
# @title scheduler
from torch.optim.lr_scheduler import LambdaLR
import math
def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, last_epoch=-1):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))
    return LambdaLR(optimizer, lr_lambda, last_epoch)

# total_steps=100
# base_lr, max_lr = 3e-5, 3e-4

# import torch
# model=torch.nn.Linear(2,3)
# optim = torch.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999))

# scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=20 , num_training_steps=total_steps) # https://docs.pytorch.org/torchtune/0.2/generated/torchtune.modules.get_cosine_schedule_with_warmup.html
# # scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, max_lr=max_lr, total_steps=total_steps, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=100.0, three_phase=True,)

# lr_lst=[]
# import matplotlib.pyplot as plt
# for t in range(total_steps):
#     lr=optim.param_groups[0]["lr"]
#     lr_lst.append(lr)
#     scheduler.step()
# plt.plot(lr_lst)

optim.param_groups[0]["lr"] = 1e-3

In [ ]:
optim.param_groups[0]["lr"] = 1e-3

In [ ]:
# @title train generate
import torch
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

# https://www.comet.com/site/blog/perplexity-for-llm-evaluation/
def Perplexity(logits, target): # [b,t,vocab_size], [b,t]
    log_probs = F.log_softmax(logits, dim=-1)
    nll = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1) # [b,t]
    perplexity = nll.mean().exp()
    return perplexity

import time
def strain(model, dataloader, optimizer, scheduler=None): # train function with automatic mixed precision
    start = begin = time.time()
    model.train()
    for i, x in enumerate(dataloader):
        x, y = x[:,:-1], x[:,1:]
        x, y = x.to(device), y.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            logits = model(x) #output = [batch size, trg len - 1, output dim]
            loss = F.cross_entropy(logits.flatten(0,1), y.flatten().to(int)) # [b*t,d], [b*t]
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.unscale_(optim)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        scaler.step(optimizer)
        scaler.update()
        if scheduler is not None: scheduler.step()
        if i % 100 == 0:
            print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
            print("strain",loss.item())
            print(generate(model, "this is what"))
            model.train()
            # perplexity = Perplexity(logits.detach(), y).item()
            start = begin = time.time()
        try: wandb.log({"train loss": loss.item()/len(y)})
        except NameError: pass

def generate(model, context, max_steps=64, temperature=1):
    x = encode(context)#.to(device)
    model.eval()
    for n in range(max_steps):
        with torch.no_grad():
            output = model(x) # gpt
        output = output[:,-1] # get logit for last character
        output = output/temperature
        output = F.softmax(output, dim=-1) # vocab_size to char
        ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
        x = torch.cat((x, ix), dim=1)
    completion = decode(x.squeeze(0))
    # completion = decode(x)
    return completion

# scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=400, num_training_steps=40000) # https://docs.pytorch.org/torchtune/0.2/generated/torchtune.modules.get_cosine_schedule_with_warmup.html
# import time
# start = begin = time.time()
for i in range(1):
    # train_loss = strain(model, train_loader, optim, scheduler=None)
    strain(model, train_loader, optim, scheduler=None)
    # strain(model, train_loader, optim, scheduler=scheduler)
    # print(generate(model, "this is what"))
    # print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
    # start = time.time()



0 time: 0.10722780227661133 0.10722827911376953
strain 5.53125
this is what                                                                
100 time: 6.49582839012146 0.06431513965719997
strain 3.5779857635498047
this is what seeees  ete e e eee eeeeet eeeee t eeeeete eee   eeeee      e s
200 time: 6.609358310699463 0.03288238202754538
strain 3.6150054931640625
this is whatttnthtttttttttttttttttttttttttttttttttttttttttthtthttttttttttttt
300 time: 6.5281007289886475 0.021688043873175435
strain 3.2951087951660156
this is whatttttotsttttttotttttttotottttttttttttttttttottttststtottttttotost
400 time: 6.624863386154175 0.016520857513694098
strain 3.1639251708984375
this is what     o    t e       o a          tt      o   e    so   tt  t    
500 time: 6.569174528121948 0.013112126234286798
strain 3.160106658935547
this is whatatt aottaoanottattttttotattttotatatttaaeta aetatttttttttttattoot
600 time: 6.6285412311553955 0.011029187534097427
strain 3.259089469909668
this is what                

KeyboardInterrupt: 

In [ ]:
print(generate(model, "this is what"))

## drawer

In [ ]:
# @title GRU pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class RNN(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        if out_dim is None: out_dim = in_dim
        self.num_layers = num_layers
        self.d_model = d_model
        self.emb = nn.Embedding(in_dim, d_model)
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, out_dim)

    def forward(self, x, h0=None): # rnn/gru
        x = self.emb(x)
        if h0==None: h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        # print(x.shape, h0.shape)
        x, h0 = self.rnn(x, h0)
        # out = out[:, -1, :] # out: (n, 128)
        x = self.fc(x) # out: (n, 10)
        return x, h0

hidden_size = 128 #64 128
num_layers = 3#2
try: vocab_size=train_loader.dataset.vocab_size#50
except NameError: vocab_size=50

model = RNN(vocab_size, hidden_size, vocab_size, num_layers).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)


# 128,2
# Test Loss: 6.360389362062727
# this is what ween new york is well it a sign more directly into simply shares
# 0 time: 5.910429954528809 5.910431623458862
# 64,2
# Test Loss: 6.167358561924526
# this is what bull morp on its aftant opereals of a b. the hamally plans beces
# 0 time: 5.655158996582031 5.655160903930664
# 64,1
# Test Loss: 5.823708357129778
# this is what achan agrive
#  the guinesst on of promjects cl funds that jound
# 0 time: 4.788918495178223

# Test Loss: 8.247572830745153
# this is what that has months with unfinsings lide to N by well
#  next offited
# 29 time: 3.902247905731201 4.377542002995809

# dim128 3lyr bptt25 2.8,.2
# this is whatever from the every coddane
# teets, in.]
# [Footnote 21: Any ot
# 9700 time: 4.124014616012573 0.00042511236912171237
# strain loss, ppl 1.3780442476272583 3.953125
# this is what seeming upon course in your drew
# plant now, as a less ever aff

b,t=2,5
x = torch.randint(0, vocab_size, (b,t), device=device)
h = torch.rand(num_layers, b, hidden_size, device=device)
out, h = model(x, h)
print(out.shape, h.shape)


In [ ]:
# @title bptt train, gen
import torch
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

def Perplexity(logits, target): # [b,t,vocab_size], [b,t]
    log_probs = F.log_softmax(logits, dim=-1)
    # nll = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1) # [b,t]
    nll = -log_probs.gather(dim=-1, index=target.to(torch.int64).unsqueeze(-1)).squeeze(-1) # [b,t]
    perplexity = nll.mean().exp()
    return perplexity

import time
def strain(model, dataloader, optim, scheduler=None, bptt=25): # train function with automatic mixed precision
    model.train()
    h0 = None
    start = begin = time.time()
    for i, x in enumerate(dataloader):
        x = x.to(device)
        xs, ys = torch.split(x[:,:-1], bptt, dim=1), torch.split(x[:,1:], bptt, dim=1)
        for (x, y) in zip(xs, ys): # https://discuss.pytorch.org/t/how-to-train-a-many-to-many-lstm-with-bptt/13005/10
            with torch.autocast(device_type=device, dtype=torch.bfloat16):
                pred, h0 = model(x, h0)
                loss = F.cross_entropy(pred.flatten(0,1), y.flatten().to(int)) # [b*t,d], [b*t]
                # loss = F.cross_entropy(pred.flatten(0,1), y.flatten()) # [b*t,d], [b*t]
            optim.zero_grad()
            scaler.scale(loss).backward()
            # scaler.unscale_(optim)
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            scaler.step(optim)
            scaler.update()
            h0 = h0.detach()

        # if scheduler is not None: scheduler.step()
        if i % 100 == 0:
            perplexity = Perplexity(pred.detach(), y).item()
            print("strain loss, ppl",loss.item(), perplexity)
            print(generate(model, "this is what"))
            model.train()
            print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
            start = begin = time.time()
        try: wandb.log({"train loss": loss.item()/len(y)})
        except NameError: pass

def generate(model, context, max_steps=64, temperature=1):
    x = encode(context)#.to(device)
    model.eval()
    hidden=None # rnn 1/3
    for n in range(max_steps):
        with torch.no_grad():
            # output = model(x) # gpt
            output, hidden = model(x, hidden) # rnn 2/3
        hidden = hidden[:,-1].unsqueeze(1) # rnn 3/3
        output = output[:,-1] # get logit for last character
        output = output/temperature
        output = F.softmax(output, dim=-1) # vocab_size to char
        ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
        x = torch.cat((x, ix), dim=1)
    completion = decode(x.squeeze(0))
    # completion = decode(x)
    return completion

# import time
# start = begin = time.time()
for i in range(1):
    # train_loss = strain(model, train_loader, optim, scheduler=None)
    strain(model, train_loader, optim, scheduler=None)
    print(generate(model, "this is what"))
    # print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
    # start = time.time()


In [ ]:
# @title pg19
# https://console.cloud.google.com/storage/browser/deepmind-gutenberg/test
# https://storage.cloud.google.com/deepmind-gutenberg/test/10146.txt

# # !wget https://storage.googleapis.com/deepmind-gutenberg/train/10006.txt
# !mkdir ./validation/
# !wget https://storage.googleapis.com/deepmind-gutenberg/validation/42306.txt -O ./validation/42306.txt
# # with open('/content/10006.txt', 'r') as f: text = f.read()

# !gsutil -m cp -r gs://deepmind-gutenberg/train ./
!gsutil -m cp -r gs://deepmind-gutenberg/test /content
# !gsutil -m cp -r gs://deepmind-gutenberg/validation ./

# # !mv -v ~/validation/.[!.]* ~/test/
# !mv ~./validation/* ~./test
!mv ~/content/validation/* ~/content/test

# path='validation/' # 17,733,002 # 4200
# path='test/' # 41,289,101
# path='train/' # 749,219,628++


In [ ]:
# @title bytedataset
import os
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, IterableDataset

def txt_iter(filepath, chunk_size=8192):
    with open(filepath, 'rb') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk: break
            yield chunk


    # class InfiniteDataset(IterableDataset):
    #     def __init__(self, start_value=0):
    #         self.current_value = start_value

    #     def __iter__(self):
    #         while True:
    #             yield self.current_value
    #             self.current_value += 1

# class ByteDataset(Dataset):
class ByteDataset(IterableDataset):
    def __init__(self, path='train/', seq_len=129, buffer_size=1024):
        self.vocab_size = 256 # utf-8 # self.enc.n_vocab # gpt2:50257
        self.seq_len, self.buffer_size = seq_len, buffer_size  # must be ≥ seq_len
        file_list = [path+f for f in os.listdir(path)]
        random.shuffle(file_list)
        self.fileiter = iter(file_list)
        # self.process()
        self.textiter = txt_iter(next(self.fileiter))
        self.buffer = []  # token buffer
        self.fill_buffer()

    def fill_buffer(self):
        while len(self.buffer) < self.buffer_size:
            try: x = next(self.textiter)
            except StopIteration:
                self.textiter = txt_iter(next(self.fileiter))
                x = next(self.textiter)
            self.buffer.extend(x)

    # def __len__(self): return 128000 # too large will cause long load first batch
    # # fs = sum([os.path.getsize(path+file) for file in os.listdir(path)])

# import re
# def strip_gutenberg_boilerplate(text):
#     start_re = re.compile(r"\*\*\* START OF (.*?) \*\*\*", re.IGNORECASE)
#     end_re = re.compile(r"\*\*\* END OF (.*?) \*\*\*", re.IGNORECASE)
#     start_match = start_re.search(text)
#     end_match = end_re.search(text)
#     start = start_match.end() if start_match else 0
#     end = end_match.start() if end_match else len(text)
#     return text[start:end].strip()

# def normalise_whitespace(text):
#     return re.sub(r'\s+', ' ', text).strip()

    # def __getitem__(self, idx):
    def __iter__(self):
        while True:
            if len(self.buffer) < self.seq_len: self.fill_buffer()
            # if len(self.buffer) < self.seq_len: raise StopIteration
            if len(self.buffer) < self.seq_len: return
            x, self.buffer = self.buffer[:self.seq_len], self.buffer[self.seq_len:]
            # # return torch.tensor(x, dtype=torch.uint8)
            # return torch.tensor(x, dtype=torch.int32)
            yield torch.tensor(x, dtype=torch.int32)

seq_len = 129 # 128
train_data = ByteDataset('validation/', seq_len) # one line of poem is roughly 50 characters
# train_data = ByteDataset('test/', seq_len) # one line of poem is roughly 50 characters
# test_data = ByteDataset('test/', seq_len) # one line of poem is roughly 50 characters
batch_size = 64 #512
# train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=2)
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=2)
# test_loader = DataLoader(test_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# https://github.com/facebookresearch/blt/blob/main/bytelatent/tokenizers/blt_tokenizer.py#L137
# def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.uint8)#, device=device)#.unsqueeze(0)
def encode(c): return torch.tensor(list(c.encode("utf-8")), dtype=torch.int32, device=device).unsqueeze(0)
def decode(x): return bytes(x.tolist()).decode("utf-8", errors='replace') # replace ignore
# for x in train_loader:
#     break
# print(x)
# s='恋 和 したい の わ'


In [ ]:
seq_len = 2**11+1 # 128
print(seq_len)
# train_data = ByteDataset('validation/', seq_len) # one line of poem is roughly 50 characters
train_data = ByteDataset('test/', seq_len) # one line of poem is roughly 50 characters
batch_size = 64 #512
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=2)

for i, x in enumerate(train_loader):
    print(i)



In [ ]:
# @title TTLinear
# Tensor Train embedding https://arxiv.org/pdf/1901.10787
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def make_einsum(num_tensors):
    a = 97
    R = chr(a+25) # 'z'
    lhs = [chr(a)+R]
    for i in range(1, num_tensors-1):lhs.append(R+chr(a+i)+R)
    lhs.append(R+chr(a+num_tensors-1))
    return ','.join(lhs) + '->' + ''.join([chr(a+i) for i in range(num_tensors)]) # az,zbz,zcz,zd->abcd

class TTLinear(nn.Module):
    def __init__(self, in_features=None, out_features=None, rank=256, std=1):
        super().__init__()
        self.lfeat = len(in_features)
        if self.lfeat==1: lst = in_features + out_features
        elif self.lfeat>=2: lst = [i*j for i, j in zip(in_features, out_features)]
        last = len(lst)
        var = last/rank**(1/(2*(std**.5)*last))
        c=1/last
        self.params = nn.ParameterList([nn.Parameter(torch.randn(lst[0], rank).clamp(-c,c)*var),
            *[nn.Parameter(torch.randn(rank, ij, rank).clamp(-c,c)*var) for ij in lst[1:-1]],
            nn.Parameter(torch.randn(rank, lst[-1]).clamp(-c,c)*var)])
        self.einsum_str = make_einsum(last)
        self.shape = [p for ij in zip(in_features, out_features) for p in ij]
        self.permute = list(range(0, 2*self.lfeat - 1, 2)) + list(range(1, 2*self.lfeat, 2))
    def weight(self): return torch.einsum(self.einsum_str, *self.params).reshape(self.shape).permute(self.permute).flatten(0,self.lfeat-1).flatten(1)

    def forward(self, x):
        weight = self.weight()
        return x.to(weight.dtype) @ weight

def one_hot(x, in_dim):
    return torch.zeros((*x.shape,in_dim), dtype=torch.int8, device=x.device).scatter_(-1, x.unsqueeze(-1).to(int), 1)

def one_hot(x, in_dim):
    b,t = x.shape
    o = torch.zeros((b,t,in_dim), dtype=bool, device=x.device)
    o[torch.arange(b).unsqueeze(-1),torch.arange(t).unsqueeze(0),x] = True
    return o

import math
class TTEmbedding(nn.Module):
    def __init__(self, in_dim, d_model, rank=256, std=1):
        super().__init__()
        self.ttlin = TTLinear(in_dim, d_model, rank, std) # https://docs.pytorch.org/docs/stable/generated/torch.nn.Embedding.html
        self.weight = self.ttlin.weight
        self.num_classes = math.prod(in_dim)

    def forward(self, x):
        # return self.ttlin(F.one_hot(x, self.num_classes))
        return self.ttlin(one_hot(x, self.num_classes))
# self.out = lambda x: x @ self.tok_emb.weight().T  # weight tying

# # in_features=(3,4,5,6); out_features=(2,3,4,5)
# in_features=[120]; out_features=[300]
# in_features=[12]; out_features=[30]
# rank=16
# # std=.5
# lin = TTLinear(in_features, out_features, rank, std).to(device)
# # x = torch.rand(4,math.prod((3,4,5,6)))
# x = torch.rand(4,7,math.prod(in_features), device=device)
# print(lin.params[0].device)
# out = lin(x)
# print(out.shape)
# print(lin.ttlin.params[0].device)

# emb = TTEmbedding(in_features, out_features, rank).to(device)
# x = torch.randint(0, math.prod(in_features), (2, 5), device=device)
# out = emb(x)
# print(out.shape)
# print(out)

# o=lin.weight
# print(o.mean().item(), o.std().item(), o.min().item(), o.max().item())

# import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (4,4)
# # plt.hist(o.flatten().tolist(), bins=20, alpha=.5, label='context mask')
# # plt.hist(o[:100,:100].flatten().tolist(), bins=20, alpha=.5, label='context mask')
# x = torch.randn(100,100)*std
# # plt.hist(x.flatten().tolist(), bins=20, alpha=.5, label='context mask')
# plt.hist([o[:100,:100].flatten().tolist(), x.flatten().tolist()], bins=20, alpha=.5, label='context mask')
# plt.show()


In [ ]:
# @title mask translate
PAD_IDX=0
def make_src_mask(src):
    # return (src != PAD_IDX).unsqueeze(1).unsqueeze(2).to(device) # [batch_size, 1, 1, src_len]?
    return (src != PAD_IDX)[:,None,None,:].to(device) # [batch_size, 1, 1, src_len]?

# attn = attn.masked_fill(mask == 0, -1e10) # [batch, n_heads, seq_len, seq_len]
def make_trg_mask(trg):
    # trg_pad_mask = (trg != PAD_IDX).unsqueeze(1).unsqueeze(2).to(device)
    trg_pad_mask = (trg != PAD_IDX)[:,None,None,:].to(device) # [batch, 1, 1, trg_len]
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=device)).bool()
    # print('make_trg_mask', trg_pad_mask.shape, trg_sub_mask.shape) # [64, 1, 1, 10], [10, 10]
    trg_mask = trg_pad_mask & trg_sub_mask # [batch, 1, trg_len, trg_len]?
    return trg_mask

def translate(model, src_sentence):
    model.eval()
    src = de_transform(src_sentence).view(1,-1).to(device)
    num_tokens = src.shape[1]
    trg_indexes = [BOS_IDX]
    max_len = src.shape[1]+5
    for i in range(max_len):
        trg_tensor = torch.tensor(trg_indexes, dtype=torch.long, device=device).unsqueeze(0)
        src_mask, trg_mask = make_src_mask(src), make_trg_mask(trg_tensor)
        with torch.no_grad():
            output = model(src, trg_tensor, src_mask, trg_mask)
        pred_token = output.argmax(2)[:,-1].item() # batch_first=F -> ?
        trg_indexes.append(pred_token)
        if pred_token == EOS_IDX: break
    trg_tokens = torch.tensor(trg_indexes[1:-1]).flatten()
    return " ".join(en_vocab.lookup_tokens(list(trg_tokens.cpu().numpy())))

def translate_fast(model, src_sentence):
    model.eval()
    with torch.no_grad():
        src = de_transform(src_sentence).view(1,-1).to(device)
        num_tokens = src.shape[1]
        trg_indexes = [BOS_IDX]
        max_len = src.shape[1]+5
        src_mask = make_src_mask(src)
        output = model.encode(src, src_mask)
        for i in range(max_len):
            trg_tensor = torch.tensor(trg_indexes, dtype=torch.long, device=device).unsqueeze(0)
            trg_mask = make_trg_mask(trg_tensor)
            output = model.decode(src, trg_tensor, src_mask, trg_mask)
            pred_token = output.argmax(2)[:,-1].item() # batch_first=F -> ?
            trg_indexes.append(pred_token)
            if pred_token == EOS_IDX: break
        trg_tokens = torch.tensor(trg_indexes[1:-1]).flatten()
        return " ".join(en_vocab.lookup_tokens(list(trg_tokens.cpu().numpy())))

# UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
# print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu ."))

src, trg = torch.randint(0, 100, (64, 10)), torch.randint(0, 100, (64, 10))
sm = make_src_mask(src)
tm = make_trg_mask(trg)
# print(sm.shape, tm.shape) # [64, 1, 1, 10], [64, 1, 10, 10]


In [ ]:
# @title translate train test

def train(model, dataloader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for src, trg in dataloader:
        src, trg = src.to(device), trg.to(device) #trg = [batch size, trg len]
        trg_input = trg[:,:-1]
        src_mask, trg_mask = make_src_mask(src), make_trg_mask(trg_input)
        print('train', src.shape, trg.shape, src_mask.shape, trg_mask.shape)
        output = model(src, trg_input, src_mask, trg_mask) #output = [batch size, trg len - 1, output dim]
        optimizer.zero_grad()
        loss = loss_fn(output.reshape(-1, output.shape[-1]), trg[:,1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(list(dataloader))

def test(model, dataloader, loss_fn):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, trg in dataloader:
            src, trg = src.to(device), trg.to(device) #trg = [batch size, trg len]
            trg_input = trg[:,:-1]
            src_mask, trg_mask = make_src_mask(src), make_trg_mask(trg_input)
            output = model(src, trg_input, src_mask, trg_mask) #output = [batch size, trg len - 1, output dim]
            loss = loss_fn(output.reshape(-1, output.shape[-1]), trg[:,1:].reshape(-1))
            epoch_loss += loss.item()
    return epoch_loss / len(list(dataloader))

# @title run
import time

loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9) # lr=0.0001

# for epoch in range(20):
for epoch in range(1):
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, loss_fn)
    val_loss = test(model, val_loader, loss_fn)
    end_time = time.time()
    print((f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    # print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")) # A group of people standing in front of an igloo .
    # @title inference
    print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")) # A group of people stand in front of an igloo .
    print(translate(model, "Ein Koch in weißer Uniform bereitet Essen in einer Restaurantküche zu .")) # A chef in a white uniform prepares food in a restaurant kitchen .
    print(translate(model, "Zwei junge Mädchen spielen Fußball auf einem Feld. .")) # Two young girls play soccer on a field. .
    print(translate(model, "Eine Frau mit Hut und Sonnenbrille steht am Strand .")) # A woman wearing a hat and sunglasses stands on the beach .
    print(translate(model, "Zwei Freunde lachen und genießen ein Eis auf einer wunderschönen Wiese .")) # Two friends laugh and enjoy ice cream on a beautiful meadow .


In [ ]:
'''
Epoch: 1, Train loss: 5.402, Val loss: 4.186, Epoch time = 41.608s
A group of people are are are are are are in a .
Epoch: 2, Train loss: 3.898, Val loss: 3.545, Epoch time = 41.068s
A group of people are standing in a crowd of people .
Epoch: 3, Train loss: 3.353, Val loss: 3.125, Epoch time = 41.566s
A group of people standing in front of a crowd .
Epoch: 4, Train loss: 2.944, Val loss: 2.830, Epoch time = 40.756s
A group of people standing in front of a building .
Epoch: 5, Train loss: 2.630, Val loss: 2.596, Epoch time = 41.468s
A group of people standing in front of a crowd .
Epoch: 6, Train loss: 2.375, Val loss: 2.429, Epoch time = 41.023s
A group of people standing in front of a house .
Epoch: 7, Train loss: 2.166, Val loss: 2.307, Epoch time = 41.604s
A group of people stand in front of a house .
Epoch: 8, Train loss: 1.984, Val loss: 2.210, Epoch time = 40.876s
A group of people stand in front of an audience .
Epoch: 9, Train loss: 1.834, Val loss: 2.131, Epoch time = 41.496s
A group of people are standing in front of an audience .
Epoch: 10, Train loss: 1.698, Val loss: 2.079, Epoch time = 41.052s
A group of people are standing in front of an empty house .
Epoch: 11, Train loss: 1.576, Val loss: 2.038, Epoch time = 41.570s
A group of people are standing in front of an audience .
Epoch: 12, Train loss: 1.475, Val loss: 2.033, Epoch time = 41.067s
A group of people stand in front of an audience .
Epoch: 13, Train loss: 1.381, Val loss: 2.017, Epoch time = 41.576s
A group of people stand in front of an operation .
Epoch: 14, Train loss: 1.292, Val loss: 1.977, Epoch time = 40.779s
A group of people stand in front of an operation .
Epoch: 15, Train loss: 1.213, Val loss: 1.948, Epoch time = 41.461s
A group of people stand in front of an operation .
Epoch: 16, Train loss: 1.139, Val loss: 1.940, Epoch time = 40.800s
A group of people stand in front of an igloo
Epoch: 17, Train loss: 1.073, Val loss: 1.940, Epoch time = 41.533s
A group of people stand in front of an igloo
Epoch: 18, Train loss: 1.006, Val loss: 1.939, Epoch time = 40.856s
A group of people stand in front of an igloo
Epoch: 19, Train loss: 0.944, Val loss: 1.947, Epoch time = 41.345s
A group of people stand in front of an igloo
Epoch: 20, Train loss: 0.893, Val loss: 1.956, Epoch time = 40.935s
A group of people stand in front of an igloo
'''

In [ ]:
# @title bleu
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    trgs = []
    pred_trgs = []
    for datum in data:
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        pred_trgs.append(pred_trg)
        trgs.append([trg])
    return bleu_score(pred_trgs, trgs)
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)
print(f'BLEU score = {bleu_score*100:.2f}')
# 36.52, which beats the ~34 of the convolutional sequence-to-sequence model and ~28 of the attention based RNN model.

def translate_sentence_vectorized(src_tensor, src_field, trg_field, model, device, max_len=50):
    assert isinstance(src_tensor, torch.Tensor)

    model.eval()
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)
    # enc_src = [batch_sz, src_len, hid_dim]

    trg_indexes = [[trg_field.vocab.stoi[trg_field.init_token]] for _ in range(len(src_tensor))]
    # Even though some examples might have been completed by producing a <eos> token
    # we still need to feed them through the model because other are not yet finished
    # and all examples act as a batch. Once every single sentence prediction encounters
    # <eos> token, then we can stop predicting.
    translations_done = [0] * len(src_tensor)
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        pred_tokens = output.argmax(2)[:,-1]
        for i, pred_token_i in enumerate(pred_tokens):
            trg_indexes[i].append(pred_token_i)
            if pred_token_i == trg_field.vocab.stoi[trg_field.eos_token]:
                translations_done[i] = 1
        if all(translations_done):
            break

    # Iterate through each predicted example one by one;
    # Cut-off the portion including the after the <eos> token
    pred_sentences = []
    for trg_sentence in trg_indexes:
        pred_sentence = []
        for i in range(1, len(trg_sentence)):
            if trg_sentence[i] == trg_field.vocab.stoi[trg_field.eos_token]:
                break
            pred_sentence.append(trg_field.vocab.itos[trg_sentence[i]])
        pred_sentences.append(pred_sentence)
    return pred_sentences, attention

from torchtext.data.metrics import bleu_score

def calculate_bleu_alt(iterator, src_field, trg_field, model, device, max_len = 50):
    trgs = []
    pred_trgs = []
    with torch.no_grad():
        for batch in iterator:
            src = batch.src
            trg = batch.trg
            _trgs = []
            for sentence in trg:
                tmp = []
                # Start from the first token which skips the <start> token
                for i in sentence[1:]:
                    # Targets are padded. So stop appending as soon as a padding or eos token is encountered
                    if i == trg_field.vocab.stoi[trg_field.eos_token] or i == trg_field.vocab.stoi[trg_field.pad_token]:
                        break
                    tmp.append(trg_field.vocab.itos[i])
                _trgs.append([tmp])
            trgs += _trgs
            pred_trg, _ = translate_sentence_vectorized(src, src_field, trg_field, model, device)
            pred_trgs += pred_trg
    return pred_trgs, trgs, bleu_score(pred_trgs, trgs)


## old

In [ ]:
# Attention Is All You Need https://arxiv.org/pdf/1706.03762.pdf
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb
# https://colab.research.google.com/github/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb
# https://www.mihaileric.com/posts/transformers-attention-in-disguise/
# https://jalammar.github.io/illustrated-transformer/
# http://nlp.seas.harvard.edu/2018/04/03/attention.html

# position embedding <-> "vocabulary" size 100 <-> model can accept sentences up to 100 tokens long
# learned positional encoding, warm-up and cool-down steps, label smoothing

In [ ]:
# @title setup

# https://pytorch.org/tutorials/beginner/translation_transformer.html
# https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c64c91cf87c13c0e83586b8e66e4d74e/translation_transformer.ipynb

# https://github.com/pytorch/data
# %pip install portalocker
# %pip install torchdata

# Create source and target language tokenizer. Make sure to install the dependencies.
!pip install -qU torchdata torchtext
!pip install -qU spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

!git clone --recursive https://github.com/multi30k/dataset.git multi30k-dataset

# !pip list | grep torch
!pip list | grep python

import torchtext.datasets as datasets

# Load the Multi30k dataset
train_iter, valid_iter, test_iter = datasets.Multi30k(split=('train', 'valid', 'test'))

# Iterate through the dataset
for src, tgt in train_iter:
    print(f"Source: {src}")
    print(f"Target: {tgt}")
    break


In [ ]:
# @title data

from torchtext.datasets import multi30k, Multi30k
# modify the URLs for the dataset since the links to the original dataset are broken https://github.com/pytorch/text/issues/1756#issuecomment-1163664163
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

from torchtext.data.utils import get_tokenizer
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')


UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

from torchtext.vocab import build_vocab_from_iterator
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))

de_tokens = [de_tokenizer(data_sample[0]) for data_sample in train_iter]
en_tokens = [en_tokenizer(data_sample[1]) for data_sample in train_iter]

de_vocab = build_vocab_from_iterator(de_tokens, min_freq=1, specials=special_symbols, special_first=True)
en_vocab = build_vocab_from_iterator(en_tokens, min_freq=1, specials=special_symbols, special_first=True)
de_vocab.set_default_index(UNK_IDX)
en_vocab.set_default_index(UNK_IDX)

import torch

def de_transform(o):
    o=de_tokenizer(o)
    o=de_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))

def en_transform(o):
    o=en_tokenizer(o)
    o=en_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))


from torch.nn.utils.rnn import pad_sequence
# function to collate data samples into batch tensors
def collate_fn(batch): # convert a batch of raw strings into batch tensors
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(de_transform(src_sample.rstrip("\n")))
        trg_batch.append(en_transform(trg_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=PAD_IDX)
    return src_batch, trg_batch


torch.manual_seed(0)

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
batch_size = 128 # 128
train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)

# vocab_transform = {SRC_LANGUAGE:de_vocab, TRG_LANGUAGE:en_vocab}
# text_transform = {SRC_LANGUAGE:de_transform, TRG_LANGUAGE:en_transform}


In [ ]:
# !pip install -q datasets
import numpy as np
np.float_ = np.float64
np.complex_ = np.complex128
!python -m spacy download de_core_news_sm


In [ ]:
# @title hf data
import torch
from datasets import load_dataset

# Load a translation dataset (e.g., WMT English-German)
dataset = load_dataset("wmt14", "de-en")

# Access train and validation splits
train_data = dataset['train']
val_data = dataset['validation']

# # Example of accessing data
# # for example in train_data:
# #     print(example['translation'])
# batch_size = 128 # 128
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn)
# val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, collate_fn=collate_fn)



# from torchtext.datasets import multi30k, Multi30k
# # modify the URLs for the dataset since the links to the original dataset are broken https://github.com/pytorch/text/issues/1756#issuecomment-1163664163
# multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
# multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

# from torchtext.data.utils import get_tokenizer
# de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
# en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

# from spacy.tokenizer import Tokenizer
# from spacy.lang.en import English
# nlp = English()
# # Create a blank Tokenizer with just the English vocab
# tokenizer = Tokenizer(nlp.vocab)

# # Construction 2
# from spacy.lang.en import English
# nlp = English()
# tokenizer = nlp.tokenizer

import numpy as np
np.float_ = np.float64
np.complex_ = np.complex128
import spacy
en_tokenizer = spacy.load("en_core_web_sm")
de_tokenizer = spacy.load("de_core_news_sm") # https://spacy.io/models/de



UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# from torchtext.vocab import build_vocab_from_iterator
# train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))

de_tokens = [de_tokenizer(data_sample[0]) for data_sample in train_iter]
en_tokens = [en_tokenizer(data_sample[1]) for data_sample in train_iter]

# de_vocab = build_vocab_from_iterator(de_tokens, min_freq=1, specials=special_symbols, special_first=True)
# en_vocab = build_vocab_from_iterator(en_tokens, min_freq=1, specials=special_symbols, special_first=True)
# de_vocab.set_default_index(UNK_IDX)
# en_vocab.set_default_index(UNK_IDX)


print("en_tokens", en_tokens)




import torch

def de_transform(o):
    o=de_tokenizer(o)
    o=de_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))

def en_transform(o):
    o=en_tokenizer(o)
    o=en_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))


from torch.nn.utils.rnn import pad_sequence
# function to collate data samples into batch tensors
def collate_fn(batch): # convert a batch of raw strings into batch tensors
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(de_transform(src_sample.rstrip("\n")))
        trg_batch.append(en_transform(trg_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=PAD_IDX)
    return src_batch, trg_batch


torch.manual_seed(0)

# train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
# val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
batch_size = 128 # 128
train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)




In [ ]:
# @title model
import torch
import torch.nn as nn
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class PositionalEncoder(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_seq_length=512):
        super().__init__()
        self.drop = nn.Dropout(dropout)
        pe = torch.zeros(max_seq_length, d_model)
        pos = torch.arange(0, max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.drop(x + self.pe[:, : x.size(1)])

class LearntPosEnc(nn.Module): # learnt positional embeddings
    def __init__(self, d_model, dropout=0.1, max_length=512):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_length, d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, src_len = x.shape
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(device) # [batch size, src len]
        return self.drop(x + self.pos_embedding(pos))


class RoPE(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, seq_len=512):
        super().__init__()
        # theta = 1.0 / (10000 ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
        theta = 1.0 / (10000 ** (torch.arange(0, dim, 2, dtype=torch.float) / dim))
        # pos = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        pos = torch.arange(seq_len).unsqueeze(1)
        angles = pos * theta # [seq_len, 1] * [dim // 2] = [seq_len, dim // 2]
        self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1) # [seq_len, dim]

    def forward(self, x):
        batch, seq_len, dim = x.shape
        # if rot_emb.shape[0] < seq_len: print("rot_emb.shape[0] < seq_len")
        rot_emb = self.rot_emb[:seq_len, :].unsqueeze(0).expand(batch, -1, -1)
        x = x.view(batch, seq_len, dim // 2, 2)
        rot_emb = rot_emb.view(batch, seq_len, dim // 2, 2)
        # rot_x = torch.einsum('...ij,...ij->...ij', x, rot_emb)
        rot_x = x * rot_emb
        # return rot_x.view(*rot_x.shape[:-2], dim)
        return rot_x.flatten(-2)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.q = nn.Linear(d_model, d_model, bias=False)
        self.k = nn.Linear(d_model, d_model, bias=False)
        self.v = nn.Linear(d_model, d_model, bias=False)
        self.lin = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.tensor((self.head_dim,), dtype=torch.float, device=device))

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        Q = self.q(query).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        K = self.k(key).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        V = self.v(value).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        attn = Q @ K.transpose(2, 3) / self.scale # attn = torch.matmul(Q, K.transpose(2, 3)) / self.scale
        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e10) # [batch, n_heads, seq_len, seq_len]
        attention = torch.softmax(attn, dim=-1)
        x = self.drop(attention) @ V # x = torch.matmul(self.drop(attention), V)
        x = x.transpose(1, 2).reshape(batch_size, -1, self.d_model)
        x = self.lin(x)
        return x, attention


class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.RMSNorm(d_model)
        self.norm2 = nn.RMSNorm(d_model)
        self.drop = nn.Dropout(dropout)
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim), nn.ReLU(), # ReLU GELU
            nn.Dropout(dropout), nn.Linear(ff_dim, d_model)
        )

    def forward(self, src, src_mask):
        src = self.norm1(src + self.drop(self.self_attn(src, src, src, src_mask)[0]))
        src = self.norm2(src + self.drop(self.ff(src)))
        return src

class Encoder(nn.Module):
    def __init__(self, d_model, n_layers, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)])

    def forward(self, src, src_mask):
        for layer in self.layers:
            src = layer(src, src_mask)
        return src

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.RMSNorm(d_model)
        self.norm2 = nn.RMSNorm(d_model)
        self.norm3 = nn.RMSNorm(d_model)
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        self.enc_attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim), nn.ReLU(), # ReLU GELU
            nn.Dropout(dropout), nn.Linear(ff_dim, d_model)
        )
        self.drop = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        trg = self.norm1(trg + self.drop(self.self_attn(trg, trg, trg, trg_mask)[0]))
        trg = self.norm2(trg + self.drop(self.enc_attn(trg, enc_src, enc_src, src_mask)[0]))
        trg = self.norm3(trg + self.drop(self.ff(trg)))
        return trg

class Decoder(nn.Module):
    def __init__(self, d_model, n_layers, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)])

    def forward(self, trg, enc_src, trg_mask, src_mask):
        for layer in self.layers:
            trg = layer(trg, enc_src, trg_mask, src_mask)
        return trg

class Seq2Seq(nn.Module):
    def __init__(self, in_dim, out_dim, d_model=512, nhead=8, enc_layers=3, dec_layers=3, ff_dim=512, dropout=0.1):
        super().__init__()
        self.encoder = Encoder(d_model, enc_layers, nhead, ff_dim, dropout)
        self.decoder = Decoder(d_model, dec_layers, nhead, ff_dim, dropout)
        # self.pos_enc = PositionalEncoder(d_model, dropout=dropout)
        # self.pos_enc = LearntPosEnc(d_model, dropout=dropout)
        self.pos_enc = RoPE(d_model)
        self.src_tok_emb = nn.Embedding(in_dim, d_model)
        self.trg_tok_emb = nn.Embedding(out_dim, d_model)
        self.d_model = d_model
        self.lin = nn.Linear(d_model, out_dim)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, trg, src_mask=None, trg_mask=None):
        src = self.pos_enc(self.src_tok_emb(src) * math.sqrt(self.d_model))
        trg = self.pos_enc(self.trg_tok_emb(trg) * math.sqrt(self.d_model))
        enc_src = self.encoder(src, src_mask)
        output = self.decoder(trg, enc_src, trg_mask, src_mask)
        output = self.lin(output)
        return output

    def encode(self, src, src_mask=None):
        return self.encoder(self.pos_enc(self.src_tok_emb(src) * math.sqrt(self.d_model)), src_mask)

    def decode(self, trg, memory, trg_mask=None, src_mask=None):
        trg = self.decoder(self.pos_enc(self.trg_tok_emb(trg) * math.sqrt(self.d_model)), memory, trg_mask, src_mask)
        return self.lin(trg)


in_dim = 50#
out_dim = 50
model = Seq2Seq(in_dim, out_dim, d_model=512, nhead=8, enc_layers=3, dec_layers=3, ff_dim=512, dropout=0.1).to(device)


In [ ]:
# @title Attention with kvcache window
import torch
import torch.nn as nn
from torch.nn import functional as F

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

# mask = torch.rand(b,t,t)<.2
def to_midx(mask): # [b,t,t+p] bool mask
    b,t = mask.shape[:2]
    idx = mask.nonzero(as_tuple=True)
    tk = torch.stack(idx[:-1], dim=-1).to(device)
    tk = torch.cat([torch.tensor([True], device=device), (tk[:-1]!=tk[1:]).any(-1)], dim=0)
    cc = len(idx[0])
    positions = torch.arange(cc, device=device)
    last_reset_pos = torch.zeros(cc, dtype=int, device=device)
    last_reset_pos[tk] = positions[tk]
    out = positions - last_reset_pos.cummax(0).values
    c=out.max()+1
    y = torch.full((b,t,c), -1, device=device)
    y[*idx[:-1],out] = idx[-1]
    return y # [b,t,c]

class Attention(nn.Module):
    # def __init__(self, d_model, cond_dim=None, n_heads=None, d_head=8, drop=0.): # .1
    def __init__(self, query_dim, cond_dim=None, n_heads=8, d_head=8, drop=0, w=64):
        super().__init__()
        self.d_model = d_model = d_head * n_heads
        self.d_head, self.n_heads = d_head, n_heads
        self.cond_dim = cond_dim
        self.pos_enc = RoPE(d_model, base=100) # 10000
        self.q = nn.Linear(query_dim, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or query_dim, 2*d_model, bias=False)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(drop) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5
        # torch.nn.init.normal_(self.qkv.weight, std=.02)
        # torch.nn.init.normal_(self.q.weight, std=1/(math.sqrt(query_dim)+math.sqrt(d_model)))
        # torch.nn.init.normal_(self.kv.weight, std=1/(math.sqrt(cond_dim or query_dim)+math.sqrt(d_model)))
        t=128
        self.midx = (torch.arange(w, dtype=torch.int32).repeat(t,1) + torch.arange(1-w, t-w+1, dtype=torch.int32).unsqueeze(-1)).to(device) # [t,w,1]
        self.w = w

    def forward(self, x, cond=None, mask=None): # [b,t,d], [batch, num_tok, cond_dim], [b,t,t(+p)]
        b,t = x.shape[:2]
        if self.cond_dim==None: cond=x # is self attn
        # q = self.q(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        # kv = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2)#.chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]

        q = self.pos_enc(self.q(x)).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        k, v = self.kv(cond).chunk(2, dim=-1)
        kv = torch.cat([self.pos_enc(k),v], dim=-1).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2) # [batch, n_heads, T/num_tok, d_head]

        import time
        start = time.time()

        # if mask != None:
            # midx = to_midx(mask) # [b,t,w]
            # # print('mask', q.shape, mask.shape, midx.shape)
            # kv = kv[torch.arange(b)[:,None,None,None], torch.arange(self.n_heads)[None,:,None,None], midx.unsqueeze(1)] # [b,h,t,w,d]

            # b,t,d = x.shape
            # if t>len(self.midx): self.midx = (torch.arange(w, dtype=torch.int32).repeat(t,1) + torch.arange(1-w, t-w+1, dtype=torch.int32).unsqueeze(-1)).to(device) # [t,w,1]
            # midx = self.midx[:t,-min(self.w,t):] # [t,w,1]
            # kv = kv[torch.arange(b)[:,None,None,None], torch.arange(self.n_heads)[None,:,None,None], midx[None,None,...]]


        kv = F.pad(kv, (0,0,self.w-1,0)) # [b, h, t+w-1, d]
        # kv = kv.as_strided((b,self.n_heads,t,self.w,2*self.d_head), kv.stride()[:-1] + kv.stride()[-2:]) # [b,h,t,w,d] # repeat stride at w's dim
        kv = kv.as_strided((b,self.n_heads,t,self.w,2*self.d_head), kv.stride()[:-1] + kv.stride()[-2:]) # [b,h,t,w,d] # repeat stride at w's dim


            # mmask = midx>=0 # F->mask # [t,c]
            # mmask = (midx>=0).unsqueeze(0) # F->mask # [1,t,c]
        # else: kv = kv.unsqueeze(3).repeat(1,1,1,t,1) # [b,h,t,t(w),d]
        print('midx', time.time()-start)

# [batch, n_heads, T, d_head]
# [b,h,t,d] @ [b,h,d,t] = [b,h,t,t]

# [batch, n_heads, T, d_head]
# [b,h,t,1,d] @ [b,h,t,d,w] = [b,h,t,1,w]

        mk, mv = kv.chunk(2, dim=-1) # [b,h,t,w,d]
        # del kv
        # attn fwd q mk attn torch.Size([64, 8, 127, 8]) torch.Size([64, 8, 100, 64, 8])
        # print('attn fwd q mk attn', q.shape, mk.shape, kv.shape)
        # attn = q.unsqueeze(3) @ mk.transpose(-2,-1) * self.scale # [b,h,t,1,d] @ [b,h,t,d,w] = [b,h,t,1,w]
        # print('attn fwd q mk attn', q.shape, mk.shape, attn.shape, mmask.shape)
        # q mk attn [64, 8, 100, 8], [64, 8, 100, 3, 8], [64, 8, 100, 1, 3], [64, 100, 3])
        # q mk attn [64, 8, 100, 8], [64, 8, 100, 64, 8], [64, 8, 100, 1, 64], [100, 64])
# attn fwd q mk attn torch.Size([64, 8, 100, 8]) torch.Size([64, 8, 100, 3, 8]) torch.Size([64, 8, 100, 1, 3]) torch.Size([100, 3])

        attn = torch.einsum("bhtd,bhtwd->bhtw", q, mk) * self.scale
        # print('attn fwd q mk attn', q.dtype, mk.dtype, attn.dtype)

        # if mask != None:
        #     # attn = attn.masked_fill(~mmask.unsqueeze(1), -torch.finfo(attn.dtype).max) # [b,t,t]->[b,1,t,t]
        #     attn = attn.masked_fill(~mmask[:,None,:,None], -torch.finfo(attn.dtype).max) # [b,t,t]->[b,1,t,1,t]
        attention = torch.softmax(attn, dim=-1) # [b,h,t,1,w]
        # out = (self.drop(attention) @ mv).squeeze(-2) # [b,h,t,1,w]@[b,h,t,w,d]=[b,h,t,1,d]
        out = torch.einsum("bhtw,bhtwd->bhtd", self.drop(attention), mv)

        out = out.transpose(1, 2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]

# td*dt->tt t(2t+d)
# td*twd->tw t(~wd-(w+2)d)
# iff 2t>wd
# gen: xqkv, patch: ~xkv ~pq, gen: xqkv pkv
# calc xqkv, calc ~xkv ~pq, get xqkv calc pkv

# typically: gen td*td=tt
# gen t1d*twd=tw, patch pd*tpd=p


b,t,d = 64,100,512
# mask = torch.rand(b,t,t, device=device)<.2


# causal_mask = torch.tril(torch.ones((b,t,x.shape[1]), dtype=bool, device=device)) # [1,t,n_cond+n_patch] # got cond, all can attend to cond
# # mask = causal_mask * mask.unsqueeze(1) if mask!=None else causal_mask # *[b,1,t] # mask left side, tril is lower left
# mask = causal_mask * ~torch.tril(torch.ones_like(causal_mask, dtype=bool, device=device), diagonal=-64) # sliding window mask


msk = torch.rand(b,t,t)
w = 3
_, ind = torch.topk(msk, w, dim=-1, sorted=False)

# # print(ind.shape, ind)
mask = torch.zeros_like(msk, dtype=bool)
mask[torch.arange(b)[:,None,None], torch.arange(t)[None,:,None], ind] = True


# model = Attention(d).to(device) # 257 ms
model = Attention(d, w=3).to(device) # 257 ms
x = torch.rand(b,t,d, device=device)

import time
start = time.time()
# out = model(x)
out = model(x, mask=mask)
print(out.shape)
print(time.time()-start)


In [ ]:
# @title torch.profiler  as_strided unfold

import torch
b,t = 2,8
mask = torch.rand(b,t,t)<.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from torch.profiler import profile, record_function, ProfilerActivity
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        # to_midx(mask)
        midx = midx[:t,-min(w,t):] # [t,w,1]
        h=4
        d=5
        kv = torch.rand(b,h,t,d)
        kv = kv[torch.arange(b)[:,None,None,None], torch.arange(h)[None,:,None,None], midx[None,None,...]]

# print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))
print(prof.key_averages().table())

# import time
# start = time.time()

# print(time.time()-start)

x = torch.rand(b,t,t)
w = 3
_, ind = torch.topk(x, w, dim=-1, sorted=False)

# # print(c)
b,h,t,d = 2,4,8,5
k = torch.rand(b,h,t,d)
k = torch.rand(b,h,t,d)
q = torch.rand(b,h,t,d)

k = F.pad(k, (0, 0, w - 1, 0))  # (B, H, T + W - 1, D)
v = F.pad(v, (0, 0, w - 1, 0))
k = F.pad(x, (0,0,w-1,0)) # [b, h, t+w-1, d]

b,h,l,d = k.shape
k_strided = k.as_strided(size=(b,h,t,w,d), stride=(k.stride(0), k.stride(1), k.stride(2), k.stride(2), k.stride(3)))
v_strided = v.as_strided(size=(b,h,t,w,d), stride=(v.stride(0), v.stride(1), v.stride(2), v.stride(2), v.stride(3)))

o = k.as_strided((b,t,w,d), k.stride()[:-1] + k.stride()[-2:]) # repeat stride at w's dim

attn_scores = torch.einsum("bhtd,bhtwd->bhtw", q, k_strided) / math.sqrt(d)
attn_weights = torch.softmax(attn_scores, dim=-1)
out = torch.einsum("bhtw,bhtwd->bhtd", attn_weights, v_strided)

# # b,h,t,d = 2,4,8,5
# k = torch.rand(b,h,t,d)
# # # print(k.stride(0), k.stride(1), k.stride(2), k.stride(2), k.stride(3))
# print(k.stride())
# # print(torch.rand(2,3,5,7).stride())
# # 3*5*7,5*7,7,1

import torch
import torch.nn.functional as F

x = torch.rand(2,3,5)
print(x)
# # o = torch.as_strided(x, (2, 2), (1, 2))
# o = torch.as_strided(x, (2, 5), (1, 3))
# print(o)
# o = torch.as_strided(x, (2, 5), (2, 3))
b,t,d = x.shape
w=5
k = F.pad(x, (0,0,w-1,0)) # [b, t, t+w-1, d]
print(k.shape)
# o = k.as_strided(size=(b,t,w,d), stride=(k.stride(0), k.stride(1), k.stride(1), k.stride(2)))
# o = k.as_strided((b,t,w,d), k.stride()[:-1] + k.stride()[-2:]) # repeat stride at w's dim
# o = k.as_strided((b,h,t,w,d), k.stride()[:-1] + k.stride()[-2:]) # repeat stride at w's dim

# o = k.view(b*h,t+w-1,d).unfold(dimension=1, size=w, step=1).view(b,h,t,w,d)
# o = k.view(b,t+w-1,d).unfold(dimension=1, size=w, step=1).view(b,t,w,d)
# o = k.unfold(dimension=1, size=w, step=1)#.view(b,t,w,d)
o = k.unfold(dimension=-2, size=w, step=1).transpose(-2,-1)

print(o.shape, o)


